In [2]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 색온도 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('./../cct_swr_test.csv', delimiter=',', dtype='float')

print(np.shape(import_data))

x_data = import_data[:, :25]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
test_x = x_data[85:, :25]
x_data = x_data[:85, :25]

y_data = []
test_y = []

for i in range(85):
    temp = []
    temp.append(import_data[i][25])
    y_data.append(temp)

raw_y = y_data
    
for i in range(85, len(import_data)):
    temp = []
    temp.append(import_data[i][25])
    test_y.append(temp)
    
# raw_y = y_data

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 25])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

W = tf.Variable(tf.random_normal([25, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(50001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 5000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
        if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_x) * 100))

(86, 26)
0 3.9184246
5000 0.652358
10000 0.5799372
15000 0.5408266
20000 0.5136056
25000 0.4926318
30000 0.4755632
35000 0.461185
40000 0.448778
45000 0.43787923


KeyboardInterrupt: 